In [1]:
import os
import sys

print(os.getcwd())
cwd0 = '../config/'
sys.path.append(cwd0)

import visualID_Eng as vID
from visualID_Eng import color
vID.init(cwd0)
import tools4PytChem as t4p

/home/romuald/ENSEIGNEMENT/0-JupyterNotebooks/PytChem/UnderDevelopment


**Start at:** Thursday 09 February 2023, 14:35:12  
**Hostname:** insa-12842 (Linux)

<p style="text-align: center"><img width="800px" src="../config/svg/PytChemBanner.svg" style="margin-left:auto; margin-right:auto"/></p>

# VUV simulation from TDDFT calculations

## Simple version (BIPS in MeCN case)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.constants as sc

def epsi(lambdaX,lambdai,fi,sigma):
    '''
    input:
        - lambdaX = wavelength
        - lambdai = vertical excitation wavelength for ith state
        - fi = oscillator strength for state i (dimensionless)
        - sigma = half-width of the Gaussian band
    output :
        gaussian distribution
    '''
    epsi = (fi / (sigma * np.sqrt(2*np.pi))) * np.exp(-0.5*((lambdaX - lambdai)/sigma)**2)
    return epsi

In [ ]:
istateN,stateN,wavelN,feN,SSqN = np.genfromtxt("./BIPS/N-MeCN-TDDFT-LBS_Te_fe.csv",skip_header=1,dtype="<U20,<U20,float,float,<U20",unpack=True)
istateMC,stateMC,wavelMC,feMC,SSqMC = np.genfromtxt("./BIPS/MC-MeCN-TDDFT-LBS_Te_fe.csv",skip_header=1,dtype="<U20,<U20,float,float,<U20",unpack=True)
sigma = 15

lambdaL = np.arange(200,601,1)
sumfN = np.zeros(len(lambdaL))
sumfMC = np.zeros(len(lambdaL))
for l in lambdaL:
    for i in range(len(wavelN)):
        sumfN[np.argwhere(l==lambdaL)[0][0]] += epsi(l,wavelN[i],feN[i],sigma)
        sumfMC[np.argwhere(l==lambdaL)[0][0]] += epsi(l,wavelMC[i],feMC[i],sigma)

In [ ]:
plt.figure(figsize=(12,8))
plt.xlim(200,600)
for i in range(len(wavelN)):
    plt.vlines(x = wavelN[i], ymin = 0, ymax = epsi(wavelN[i],wavelN[i],feN[i],sigma), colors = 'red')
plt.xlabel('wavelength / nm',size=14,fontweight='bold',color='blue')
plt.ylabel('fe',size=14,fontweight='bold',color='blue')
plt.xticks(fontsize=16,fontweight='bold')
plt.yticks(fontsize=16,fontweight='bold')
plt.show()

plt.figure(figsize=(12,8))
plt.xlim(200,600)
for i in range(len(wavelMC)):
    plt.vlines(x = wavelMC[i], ymin = 0, ymax = epsi(wavelMC[i],wavelMC[i],feMC[i],sigma), colors = 'red')
plt.xlabel('wavelength / nm',size=14,fontweight='bold',color='blue')
plt.ylabel('fe',size=14,fontweight='bold',color='blue')
plt.xticks(fontsize=16,fontweight='bold')
plt.yticks(fontsize=16,fontweight='bold')
plt.show()

In [ ]:
plt.figure(figsize=(12,8))
for i in range(len(wavelN)):
    plt.vlines(x = wavelN[i], ymin = 0, ymax = epsi(wavelN[i],wavelN[i],feN[i],sigma), colors = 'red')
for i in range(len(wavelN)):
    plt.vlines(x = wavelMC[i], ymin = 0, ymax = epsi(wavelMC[i],wavelMC[i],feMC[i],sigma), colors = 'blue')
plt.xlabel('wavelength / nm',size=14,fontweight='bold',color='blue')
plt.ylabel('fe',size=14,fontweight='bold',color='blue')
plt.xticks(fontsize=16,fontweight='bold')
plt.yticks(fontsize=16,fontweight='bold')
plt.plot(lambdaL,sumfN,color='purple',linewidth=2,label="N")
plt.plot(lambdaL,sumfMC,color='black',linewidth=2,linestyle='--',label="MC")
plt.xlim(250,600)
plt.legend()
plt.show()

## Test case (Gaussian website Whitepaper)

$$\varepsilon_{i}(\bar{\nu})=\varepsilon_{i}^{\mathrm{max}}\exp\left[-\left(\frac{\bar{\nu}-\bar{\nu}_{i}}{\sigma}\right)^{2}\right]$$
It is demonstrated, in a [Gaussian whitepaper](https://gaussian.com/uvvisplot/), that the previous equation becomes, in the cgs units system:
$$\varepsilon_{i}(\bar{\nu})=\frac{\sqrt{\pi}e^{2}N_{\mathrm{A}}}{1000\ln(10)c^{2}m_{e}}\frac{f_{i}}{\sigma}\exp\left[-\left(\frac{\bar{\nu}-\bar{\nu}_{i}}{\sigma}\right)^{2}\right]$$

where:
- $\varepsilon_i$ is in units of L∙mol$^{-1}$∙cm$^{-1}$
- $f_i$ is the oscillator strength
- $\sigma$, the half-width of the Gaussian band at each $\varepsilon^{\mathrm{max}}$, is in cm$^{-1}$
- $c$ is the speed of light in cm∙s$^{-1}$
- $N_\mathrm{A}$ is the Avogadro number, in mol$^{-1}$
- $m_e$ is the electron mass in kg
- $e$ is the elementary charge, in electrostatic unit of charge (esu)
- $f_i$ is the dimensionless oscillator strength

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.constants as sc

c = sc.c*1e2 #cm-1
NA = sc.N_A #mol-1
me = sc.m_e*1000 #g
e = sc.e*sc.c*10 #elementary charge in esu 
print(f"c = {c:.2e} cm.s-1.   NA = {NA:.3e} mol-1.  me = {me:.2e} g.  e = {e:.2e} esu")
#calculation of the pre-factor
pf = np.sqrt(np.pi)*e**2*NA/(1000*np.log(10)*c**2*me)
print(f"pf = {pf:8e}")

sigma = 0.4 #eV this the default value in GaussView
eV2cm1 = sc.e / (sc.h * sc.c *100)
eV2nm = (sc.h * sc.c)*1e9 / (sc.e)
print("eV to cm-1",eV2cm1)
print("eV to nm",eV2nm)

sigmanm = eV2nm/sigma
sigmabar = 1e7 / sigmanm
print("sigma = ", sigma ,"eV ->",sigmanm," nm. sigmabar = ", sigmabar)

def epsi(lambdaX,lambdai,fi,sigma):
    '''
    input:
        - lambdaX = wavelength variable, in nm
        - lambdai = vertical excitation wavelength for ith state, in nm
        - fi = oscillator strength for state i (dimensionless)
        - sigma = half-width of the Gaussian band, in nm
    output :
        molar absorption coefficient, in L mol-1 cm-1
    '''
    import scipy.constants as sc
    c = sc.c*1e2 #cm-1
    NA = sc.N_A #mol-1
    me = sc.m_e*1000 #g
    e = sc.e*sc.c*10 #elementary charge in esu 
    pf = np.sqrt(np.pi)*e**2*NA/(1000*np.log(10)*c**2*me)
    nubarX = 1e7 / lambdaX # nm to cm-1
    nubari = 1e7 / lambdai
    sigmabar = 1e7 / sigma 
    epsi = pf * (fi / sigmabar) * np.exp(-((nubarX - nubari)/sigmabar)**2)
    return epsi

In [ ]:
wavel = np.array([154.07,192.80,284.45])
print(1e7/sigmanm,1e7/wavel)
fe = [0.0061,0.5035,0.0002]
plt.figure(figsize=(12,8))
for i in range(len(wavel)):
    plt.vlines(x = wavel[i], ymin = 0, ymax = fe[i], colors = 'red')
plt.xlabel('wavelength / nm',size=14,fontweight='bold',color='blue')
plt.ylabel('oscillator strength',size=14,fontweight='bold',color='blue')
plt.xticks(fontsize=16,fontweight='bold')
plt.yticks(fontsize=16,fontweight='bold')
plt.show()

plt.figure(figsize=(12,8))
for i in range(len(wavel)):
    plt.vlines(x = wavel[i], ymin = 0, ymax = epsi(wavel[i],wavel[i],fe[i],sigmanm), colors = 'red')
plt.xlabel('wavelength / nm',size=14,fontweight='bold',color='blue')
plt.ylabel('molar absorption coefficient / L mol$^{-1}$ cm$^{-1}$',size=14,fontweight='bold',color='blue')
plt.xticks(fontsize=16,fontweight='bold')
plt.yticks(fontsize=16,fontweight='bold')
plt.show()

In [ ]:
lambdaL = np.arange(100,301,1)
sumf = np.zeros(len(lambdaL))
k = -1
for l in lambdaL:
    k += 1
    for i in range(len(wavel)):
        sumf[np.argwhere(l==lambdaL)[0][0]] += epsi(l,wavel[i],fe[i],sigmanm)


In [ ]:
plt.figure(figsize=(12,8))
for i in range(len(wavel)):
    plt.vlines(x = wavel[i], ymin = 0, ymax = epsi(wavel[i],wavel[i],fe[i],sigmanm), colors = '#0000ff')
plt.xlabel('wavelength / nm',size=14,fontweight='bold',color='blue')
plt.ylabel('molar absorption coefficient / L mol$^{-1}$ cm$^{-1}$',size=14,fontweight='bold',color='blue')
plt.xticks(fontsize=16,fontweight='bold')
plt.yticks(fontsize=16,fontweight='bold')
plt.plot(lambdaL,sumf,linewidth=3,color="#00007f")
plt.show()

<div class="rq">

Same as on the [Gaussian16](https://gaussian.com/uvvisplot/) website. Yes!
</div>

In [29]:
vID.end(cwd0)

**End at:** Wednesday 08 February 2023, 18:16:54  
**Duration:** 01:12:05 952ms

<p style="text-align: center"><img width="800px" src="../config/svg/logoEnd.svg" style="margin-left:auto; margin-right:auto"/></p>